In [1]:
import os
import tensorflow as tf

In [2]:
#import image data generator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#define the path and split your dataset into training and validation sets
dataset=tf.keras.preprocessing.image_dataset_from_directory("/content/drive/MyDrive/Dataset")
ds_train=tf.keras.preprocessing.image_dataset_from_directory("/content/drive/MyDrive/Dataset",validation_split=0.2,subset="training",seed=123,image_size=(224,224))
ds_val=tf.keras.preprocessing.image_dataset_from_directory("/content/drive/MyDrive/Dataset",validation_split=0.2,subset="validation",seed=123,image_size=(224,224))

Found 2300 files belonging to 2 classes.
Found 2300 files belonging to 2 classes.
Using 1840 files for training.
Found 2300 files belonging to 2 classes.
Using 460 files for validation.


In [18]:
trainPath = r"/content/body"
testPath = r"/content/level"

In [14]:
!unzip /content/body-20241107T091636Z-001.zip -d /content/body

Archive:  /content/body-20241107T091636Z-001.zip
  inflating: /content/body/body/validation/02-side/0027.JPEG  
  inflating: /content/body/body/validation/02-side/0037.JPEG  
  inflating: /content/body/body/validation/02-side/0017.jpeg  
  inflating: /content/body/body/validation/02-side/0025.JPEG  
  inflating: /content/body/body/validation/02-side/0047.JPEG  
  inflating: /content/body/body/validation/02-side/0041.JPEG  
  inflating: /content/body/body/validation/02-side/0030.jpeg  
  inflating: /content/body/body/validation/02-side/0029.JPEG  
  inflating: /content/body/body/validation/02-side/0032.JPEG  
  inflating: /content/body/body/validation/02-side/0046.JPEG  
  inflating: /content/body/body/validation/02-side/0018.JPEG  
  inflating: /content/body/body/validation/02-side/0024.JPEG  
  inflating: /content/body/body/validation/02-side/0039.JPEG  
  inflating: /content/body/body/validation/02-side/0044.JPEG  
  inflating: /content/body/body/validation/02-side/0016.JPEG  
  infl

In [17]:
!unzip /content/level-20241107T091218Z-001.zip -d /content/level

Archive:  /content/level-20241107T091218Z-001.zip
  inflating: /content/level/level/validation/03-severe/0028.JPEG  
  inflating: /content/level/level/validation/03-severe/0025.JPEG  
  inflating: /content/level/level/validation/03-severe/0035.JPEG  
  inflating: /content/level/level/validation/01-minor/0008.jpeg  
  inflating: /content/level/level/validation/03-severe/0034.JPEG  
  inflating: /content/level/level/validation/03-severe/0032.JPEG  
  inflating: /content/level/level/validation/01-minor/0019.JPEG  
  inflating: /content/level/level/validation/03-severe/0026.JPEG  
  inflating: /content/level/level/validation/03-severe/0019.JPEG  
  inflating: /content/level/level/validation/03-severe/0017.JPEG  
  inflating: /content/level/level/validation/01-minor/0009.jpeg  
  inflating: /content/level/level/validation/03-severe/0027.JPEG  
  inflating: /content/level/level/validation/01-minor/0010.JPEG  
  inflating: /content/level/level/validation/01-minor/0012.JPEG  
  inflating: /con

In [19]:
print("Checking training directory:", trainPath)
if os.path.exists(trainPath):
    print("Training directory exists. Contents:")
    for item in os.listdir(trainPath):
        print(item)
else:
    print("Training directory does not exist.")

print("\nChecking testing directory:", testPath)
if os.path.exists(testPath):
    print("Testing directory exists. Contents:")
    for item in os.listdir(testPath):
        print(item)
else:
    print("Testing directory does not exist.")

Checking training directory: /content/body
Training directory exists. Contents:
body

Checking testing directory: /content/level
Testing directory exists. Contents:
level


**Image** **data** **augmentation**

In [20]:
# Setting parameters for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

**For** **Body** **Damage**

In [21]:
# Flow from directory
training_set = train_datagen.flow_from_directory(
    trainPath,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)
test_set = val_datagen.flow_from_directory(
    testPath,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

Found 1150 images belonging to 1 classes.
Found 1150 images belonging to 1 classes.


**For** **the** **level** **of** **damage**

In [22]:
# Flow from directory
training_set = train_datagen.flow_from_directory(
    trainPath,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)
test_set = val_datagen.flow_from_directory(
    testPath,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

Found 1150 images belonging to 1 classes.
Found 1150 images belonging to 1 classes.


In [23]:
#Importing model building libraries
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

# Define imageSize
imageSize = (224, 224) # Assuming you are using 224x224 images


In [24]:
#Adding preprocessing layer to the front of vgg
vgg=VGG16(input_shape=list(imageSize) + [3], weights='imagenet',include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [25]:
for layer in vgg.layers:
  layer.trainable=False

In [26]:
x=Flatten()(vgg.output)

In [27]:
#Adding output layer
prediction = Dense(4, activation='softmax')(x)

In [28]:
#create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [29]:
#view the structure of model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       100,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,815,044 (56.51 MB)

 Trainable params: 100,356 (392.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [30]:
#compiling the cnn model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [31]:
# Compile the model with sparse_categorical_crossentropy
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [32]:
import sys
# Fit the model
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch = 979//10,
    validation_steps=171//10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 708s 7s/step - accuracy: 0.9473 - loss: 0.1153 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
18/97 ━━━━━━━━━━━━━━━━━━━━ 8:10 6s/step - accuracy: 1.0000 - loss: 0.0000e+00

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


97/97 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 704s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 702s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 702s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 263s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 741s 8s/step - accuracy: 1.0

In [33]:
#save the model
model.save('body.h5')

In [34]:
#save the model
model.save('level.h5')

In [35]:
model1 = Model(inputs=vgg.input, outputs=prediction)

In [36]:
model1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       100,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,815,044 (56.51 MB)

 Trainable params: 100,356 (392.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [37]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [38]:
model1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [42]:
import sys
# Fit the model

r = model1.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch = 979//10,
    validation_steps=171//10
)



Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 705s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 743s 8s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 737s 7s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 742s 8s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - ac

In [43]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [44]:
model=load_model('level.h5')

In [45]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [46]:
model=load_model('body.h5')

In [47]:
import numpy as np

In [48]:
#creating a function used to predict, so need some preprocessing
def detect(frame):
  img = cv2.resize(frame, (224, 224))  # Resize to the input size expected by the model
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  # scaling need to be done
  if(np.max(img) > 1):
    img = img / 255.0
  img = np.array([img])  # then to array format
  prediction = model.predict(img)
  print(prediction)
  # Update label list to include 4 classes
  label = ["minor", "moderate", "severe", "additional_class"]
  preds = label[np.argmax(prediction)]
  return preds

In [62]:
import os
data=r"/content/drive/MyDrive/Dataset/body-20241208T052403Z-001/body/training/00-front/0016.JPEG"

if not os.path.exists(data):
    raise FileNotFoundError(f"Image file not found at: {data}")

image = cv2.imread(data)
print(detect(image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 549ms/step
[[1.1514104e-22 1.0000000e+00 6.7861147e-23 4.6294786e-22]]
moderate


In [61]:
import os
data=r"/content/drive/MyDrive/Dataset/body-20241208T052403Z-001/body/validation/01-rear/0019.JPEG"

if not os.path.exists(data):
    raise FileNotFoundError(f"Image file not found at: {data}")

image = cv2.imread(data)
print(detect(image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 973ms/step
[[3.6876048e-23 1.0000000e+00 1.7811085e-23 1.4303757e-22]]
moderate


In [59]:

import os
data=r"/content/drive/MyDrive/Dataset/body-20241208T052403Z-001/body/validation/02-side/0001.JPEG"

if not os.path.exists(data):
    raise FileNotFoundError(f"Image file not found at: {data}")

image = cv2.imread(data)
print(detect(image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
[[1.4351134e-23 1.0000000e+00 6.0308132e-24 3.5641285e-23]]
moderate
